<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_training_vgg16_bn_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%time
!rm -r /content/cloned-repo
%cd ..
!git clone -l -s git://github.com/kavyajeetbora/EAST.git /content/cloned-repo
%cd /content/cloned-repo
!ls

/
Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 160 (delta 44), reused 0 (delta 0), pack-reused 82
Receiving objects: 100% (160/160), 6.44 MiB | 10.68 MiB/s, done.
Resolving deltas: 100% (89/89), done.
/content/cloned-repo
dataset.py		    EAST_resnet.py		    loss.py
detect.py		    EAST_training.ipynb		    model.py
dsiplay_ground_truth.ipynb  EAST_training_vgg16_bn_2.ipynb  model_resnet.py
EAST_detection.ipynb	    EAST_training_vgg16_bn.ipynb    train.py
CPU times: user 81.4 ms, sys: 26.4 ms, total: 108 ms
Wall time: 7.18 s


In [0]:
from dataset import *

In [5]:
# AI4Bharat dataset
image_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Images/Images.Zip (Unzipped Files)'
label_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Annotations/Annotations.Zip (Unzipped Files)'

image_files = sorted(os.listdir(image_folder))
annotation_files = sorted(os.listdir(label_folder))
print('Total files in Image folder',len(image_files), '| Total files in Annotation folder',len(annotation_files))

training_dataset = custom_dataset(image_folder, label_folder)
print(len(training_dataset))

Total files in Image folder 5000 | Total files in Annotation folder 5000
5000


## Dataset Initial Check

In [6]:
index = np.random.randint(low=1,high=len(image_files)+1)
img = Image.open(os.path.join(image_folder,image_files[index]))
label = os.path.join(label_folder,annotation_files[index])
with open(label, 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)
  
print(labels)

[-265.96959777  249.44606851  -22.02601272  257.93051221  -25.05641461
  384.4998741  -270.46745674  375.92892847]
--------------------
[-122.58369414  165.04793694  -35.91729904  168.68283167  -37.12909739
  214.29963057 -125.26294957  210.5782339 ]
--------------------
[-30.04747074 169.0288394  105.04500788 175.5182979  102.19289726
 223.98603338 -31.51855187 219.05179607]
--------------------
[109.36095152 177.24652295 196.02734662 180.88141769 193.43451878
 224.9429954  108.23558076 221.39460259]
--------------------
[171.84755249 316.52078733 202.32025274 299.15747563 242.02439734
 350.13376714 211.46526949 368.9657981 ]
--------------------
[-258.62325224  124.60493167 -123.44434603  129.62567091 -126.55573943
  182.49956416 -260.18076097  176.0966076 ]
--------------------
[ 72.40980573 254.65432032  96.32125693 248.69475495 106.33236165
 278.76115554  80.86702579 286.10293824]
--------------------
[ 81.21273616 280.22806121 106.50521684 275.82371703 116.42989397
 307.35883688 

In [7]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(477.)

## Loss for one batch

In [8]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=12)
imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))
print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())
print(torch.sum(gt_score))

torch.Size([12, 3, 512, 512]) torch.Size([12, 1, 128, 128]) torch.Size([12, 5, 128, 128]) torch.Size([12, 1, 128, 128])
tensor(5795.)
CPU times: user 4.96 s, sys: 6.68 s, total: 11.6 s
Wall time: 12.7 s


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
%%time
from model import EAST
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_vgg16.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)
imgs, gt_score, gt_geo, ignored_maps = imgs.to(device), gt_score.to(device), gt_geo.to(device), ignored_maps.to(device)
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([12, 1, 128, 128]) torch.Size([12, 5, 128, 128])
CPU times: user 4.05 s, sys: 845 ms, total: 4.89 s
Wall time: 4.84 s


In [11]:
from loss import Loss

loss_fn = Loss()
loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.67216349, angle loss is 0.03908067, iou loss is 0.68666452


tensor(1.7496, device='cuda:0', grad_fn=<AddBackward0>)

## Training Full Batch 

fine tuning pretrained model [east_vgg16.pth](https://drive.google.com/file/d/1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg/view)

In [0]:
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_vgg16.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    #scheduler = lr_scheduler.StepLR(optimizer, step_size=epoch_iter, gamma=0.1)
    loss_curve = []
    
    for epoch in range(epoch_iter):
      model.train()
      #scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      
      
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, 'model_epoch_{}.pth'.format(epoch+1))
      
      ## plotting the loss curve
      loss_curve.append(epoch_loss/int(file_num/batch_size))
    return loss_curve

In [0]:
%%time
loss_curve = train(model, image_folder, label_folder, batch_size=2, lr=1e-4, epoch_iter=10, interval = 1)

5000
classify loss is 0.83189690, angle loss is 0.01564349, iou loss is 0.66675836
Epoch is [1/10], mini-batch is [1/2500], time consumption is 0.41986227, batch_loss is 1.65509009
classify loss is 0.91481495, angle loss is 0.04153585, iou loss is 0.70476246
Epoch is [1/10], mini-batch is [2/2500], time consumption is 0.33698487, batch_loss is 2.03493595
classify loss is 0.68110538, angle loss is 0.02378340, iou loss is 0.47851628
Epoch is [1/10], mini-batch is [3/2500], time consumption is 0.31375933, batch_loss is 1.39745569
classify loss is 0.87845224, angle loss is 0.07874695, iou loss is 0.91707355
Epoch is [1/10], mini-batch is [4/2500], time consumption is 0.33422780, batch_loss is 2.58299541
classify loss is 0.30888748, angle loss is 0.01639923, iou loss is 0.38811854
Epoch is [1/10], mini-batch is [5/2500], time consumption is 0.29674673, batch_loss is 0.86099827
classify loss is 0.99999917, angle loss is 0.16491562, iou loss is 1.08096695
Epoch is [1/10], mini-batch is [6/250

In [0]:
plt.plot(loss_curve)

[Adam and AdamW algorithms](https://www.fast.ai/2018/07/02/adam-weight-decay/)